In [1]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
df = pd.read_csv("Mouser_source_20240910.tsv", sep='\t', header=None)
df.columns = ['Part Number']

In [4]:
df.head()

,Part Number
0,PartNumber
1,0 048 71
2,0 371 04
3,0 371 05
4,0 371 09


In [5]:
df.shape

(402850, 1)

In [5]:
url = "https://www.mouser.com/"
driver_path = r'chromedriver.exe'
chrome_options = Options()
count = 0

In [14]:
import gzip
import json
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)
driver.get(url)

# Load and add cookies from the JSON file
with open('cookies.json', 'r') as file:
    cookies = json.load(file)

for cookie in cookies:
    if 'expiry' in cookie:
        del cookie['expiry']
    if 'sameSite' in cookie and cookie['sameSite'] not in ["Strict", "Lax", "None"]:
        del cookie['sameSite']
    try:
        driver.add_cookie(cookie)
    except Exception as e:
        print(f"Error adding cookie: {cookie['name']}, Error: {e}")

for x in df['Part Number']:
    count += 1
    driver.get(url)

    time.sleep(2)

    # Interact with the search bar and button
    search_bar = driver.find_element("xpath", '//input[@data-testid="global-search"]')
    search_bar.send_keys(x)

    search_button = driver.find_element("xpath", '//button[@data-testid="hdrSrch"]')
    search_button.click()

    time.sleep(2)

    # Save the first search result page (Home Page) as a compressed HTML file
    home_page_folder = os.path.join(os.getcwd(), "HomePage")

    if not os.path.exists(home_page_folder):
        os.makedirs(home_page_folder)

    # Define a path for the compressed HTML file with the part number as the name
    home_html_file_path = os.path.join(home_page_folder, f"{x}.html.gz")

    # Save the current page as a compressed HTML file
    with gzip.open(home_html_file_path, 'wt', encoding='utf-8') as file:
        file.write(driver.page_source)

    # Now check for the product number in search results and navigate if found
    try:
        element = driver.find_element(By.ID, "searchResultsTbl")
        part_number_elements = driver.find_elements(By.XPATH, '//div[@class="mfr-part-num hidden-xs"]//a')

        for part_element in part_number_elements:
            part_text = part_element.text.strip()

            if part_text == x:
                part_url = part_element.get_attribute("href")
                driver.get(part_url)

                time.sleep(2)

                # Save the detailed product page (Detail Page) as a compressed HTML file
                detail_page_folder = os.path.join(os.getcwd(), "DetailPage")

                if not os.path.exists(detail_page_folder):
                    os.makedirs(detail_page_folder)

                # Define a path for the compressed HTML file with the part number as the name
                detail_html_file_path = os.path.join(detail_page_folder, f"{x}.html.gz")

                # Save the detailed page as a compressed HTML file
                with gzip.open(detail_html_file_path, 'wt', encoding='utf-8') as file:
                    file.write(driver.page_source)

                break

    except:
        pass

    print(f'{count} parts done.')

driver.quit()


164 parts done.


KeyboardInterrupt: 